In [2]:
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

In [26]:
hk_tickers = pd.read_csv('HKTickers.csv')

In [107]:
count=0
selected_keys=['symbol', 'sector', 'industry', 'volume', 'profitMargins', 'shortName', 'longName', 'exchange']
tickersFrame = pd.DataFrame(data=[],columns=selected_keys)

for t in hk_tickers.Tickers:
    count+=1
    if (count<10):
        info = yf.Ticker(t).info
        s={key: info[key] for key in selected_keys}
        tickersFrame=tickersFrame.append(s, ignore_index=True)

In [109]:
tickersFrame.set_index(['symbol'])

,sector,industry,volume,profitMargins,shortName,longName,exchange
symbol,,,,,,,
0001.HK,Industrials,Conglomerates,5192567,0.13379,CKH HOLDINGS,CK Hutchison Holdings Limited,HKG
0002.HK,Utilities,Utilities—Regulated Electric,2249689,0.06145,CLP HOLDINGS,CLP Holdings Limited,HKG
0003.HK,Utilities,Utilities—Regulated Gas,14829614,0.21171,HK & CHINA GAS,The Hong Kong and China Gas Company Limited,HKG
0004.HK,Real Estate,Real Estate Services,2853476,0.29174,WHARF HOLDINGS,Wharf (Holdings) Limited,HKG
0005.HK,Financial Services,Banks—Diversified,15047538,0.27508,HSBC HOLDINGS,HSBC Holdings plc,HKG
0006.HK,Utilities,Utilities—Independent Power Producers,1986603,5.03584,POWER ASSETS,Power Assets Holdings Limited,HKG
0007.HK,Industrials,Conglomerates,2082000,-2.66544,HK FINANCE INV,Hong Kong Finance Investment Holding Group Lim...,HKG
0008.HK,Communication Services,Telecom Services,4370233,0.02382,PCCW,PCCW Limited,HKG
0009.HK,Real Estate,Real Estate—Development,200,0.00000,NINE EXPRESS,Nine Express Limited,HKG


In [8]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show balance heet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show options expirations


# get option chain for specific expiration
opt = msft.option_chain('2019-12-06')
# data available via: opt.calls, opt.puts

In [48]:
msft.info

{'zip': '98052',
 'sector': 'Technology',
 'fullTimeEmployees': 144000,
 'longBusinessSummary': "Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); and Skype, Outlook.com, and OneDrive. It also provides LinkedIn that includes Talent and marketing solutions, and subscriptions; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. The company's Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also provides support services and Microsoft 

In [45]:
msft.earnings['ProfitRatio']=msft.earnings['Earnings']/msft.earnings['Revenue']

In [43]:
msft.earnings

,Revenue,Earnings,ProfitRatio
Year,,,
2016,91154000000,20539000000,0.225322
2017,96571000000,25489000000,0.263941
2018,110360000000,16571000000,0.150154
2019,125843000000,39240000000,0.311817


In [4]:
data = pdr.get_data_yahoo("MSFT", start="2010-01-01", end="2017-04-30")

[*********************100%***********************]  1 of 1 completed


In [6]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-12-31,30.980000,30.990000,30.480000,30.480000,23.990791,31929700
2010-01-04,30.620001,31.100000,30.590000,30.950001,24.360727,38409100
2010-01-05,30.850000,31.100000,30.639999,30.959999,24.368599,49749600
2010-01-06,30.879999,31.080000,30.520000,30.770000,24.219046,58182400
2010-01-07,30.629999,30.700001,30.190001,30.450001,23.967175,50559700
...,...,...,...,...,...,...
2017-04-24,67.480003,67.660004,67.099998,67.529999,64.372856,29770000
2017-04-25,67.900002,68.040001,67.599998,67.919998,64.744644,30242700
2017-04-26,68.080002,68.309998,67.620003,67.830002,64.658836,26190800
